In [ ]:
import os
import shutil
import random
import platform
import numpy as np
import matplotlib.pyplot as plt

import pyswarms.single as pso
from bayes_opt import BayesianOptimization
from sionna.rt import load_scene, PlanarArray, Transmitter, RadioMapSolver

# =======================================================
# 0. Configuración y Limpieza
# =======================================================
RESULTS_DIR = "resultados"
SCENE_FILE = "/BLENDER/DT.xml" #Ubicación archivo xml generado en blender

FREQUENCY = 3.5e9
NOISE_BW = 20e6
TX_Z = 25.0

PSO_SEEDS = [11, 22, 33, 44, 55]
BAYES_SEED = 42

def set_all_seeds(seed: int):
    np.random.seed(seed)
    random.seed(seed)

ORIGINAL_POS = {
    "tx1": [-378, 181, TX_Z],
    "tx2": [135, 116, TX_Z],
    "tx3": [8, -269, TX_Z]
}

MAP_X_MIN, MAP_X_MAX = -500, 500
MAP_Y_MIN, MAP_Y_MAX = -375, 375

if os.path.exists(RESULTS_DIR):
    shutil.rmtree(RESULTS_DIR)
os.makedirs(RESULTS_DIR)

# =======================================================
# 1. Cargar Escena y Funciones
# =======================================================
print("[1/8] Cargando Gemelo Digital y configurando antenas...")
scene = load_scene(SCENE_FILE)
scene.frequency = FREQUENCY
rm_solver = RadioMapSolver()

iso_tx_array = PlanarArray(num_rows=1, num_cols=4, vertical_spacing=0.5, horizontal_spacing=0.5, pattern="iso", polarization="V")
iso_rx_array = PlanarArray(num_rows=1, num_cols=1, vertical_spacing=0.5, horizontal_spacing=0.5, pattern="iso", polarization="V")
mimo_tx_array = PlanarArray(num_rows=4, num_cols=4, vertical_spacing=0.5, horizontal_spacing=0.5, pattern="tr38901", polarization="V")

def setup_transmitters(scene, positions):
    for name in ["tx1", "tx2", "tx3", "rx"]:
        try: scene.remove(name)
        except: pass
    scene.add(Transmitter(name="tx1", position=positions["tx1"], orientation=[0, 0, 0]))
    scene.add(Transmitter(name="tx2", position=positions["tx2"], orientation=[0, 0, 0]))
    scene.add(Transmitter(name="tx3", position=positions["tx3"], orientation=[0, 0, 0]))

def compute_sinr_map(scene, solver):
    rm = solver(scene, max_depth=8, samples_per_tx=10**7, cell_size=(1, 1), center=[0, 0, 0], size=[1000, 750], orientation=[0, 0, 0])
    rss = rm.rss.numpy()
    signal = np.max(rss, axis=0)
    interference = np.sum(rss, axis=0) - signal
    noise_power = 10**((-174 - 30)/10) * NOISE_BW * 10**(7/10)
    sinr = signal / (interference + noise_power)
    sinr_db = 10 * np.log10(sinr + 1e-15)
    return float(np.mean(sinr_db)), sinr_db

def functional_coverage(sinr_db, thr_db=0.0):
    sinr_db = np.asarray(sinr_db)
    valid = np.isfinite(sinr_db)
    denom = int(np.sum(valid))
    if denom == 0: return float("nan"), 0, 0
    num = int(np.sum((sinr_db > thr_db) & valid))
    return num/denom, num, denom

# =======================================================
# Paso 2: Simulación ULA (Posiciones Originales)
# =======================================================
print("[2/8] Evaluando escenario ULA Original...")
scene.tx_array = iso_tx_array
scene.rx_array = iso_rx_array
setup_transmitters(scene, ORIGINAL_POS)

omni_avg_sinr_orig, omni_map_orig = compute_sinr_map(scene, rm_solver)
fc_orig, _, _ = functional_coverage(omni_map_orig, thr_db=0.0)
print(f"  -> SINR Promedio: {omni_avg_sinr_orig:.2f} dB | Cobertura (>0dB): {100*fc_orig:.2f}%")

# =======================================================
# Paso 3: Optimización de Posición (PSO)
# =======================================================
print("\n[3/8] Iniciando Optimización PSO (Posición)...")

def pso_objective_function(particles):
    n_particles = particles.shape[0]
    fitness_scores = np.zeros(n_particles, dtype=float)
    scene.tx_array = iso_tx_array
    scene.rx_array = iso_rx_array

    for i in range(n_particles):
        pos = particles[i]
        scene.transmitters["tx1"].position = [float(pos[0]), float(pos[1]), TX_Z]
        scene.transmitters["tx2"].position = [float(pos[2]), float(pos[3]), TX_Z]
        scene.transmitters["tx3"].position = [float(pos[4]), float(pos[5]), TX_Z]
        avg_sinr, _ = compute_sinr_map(scene, rm_solver)
        fitness_scores[i] = -avg_sinr
    return fitness_scores

options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
bounds = (np.array([-500, -375, -500, -375, -500, -375], dtype=float), 
          np.array([ 500,  375,  500,  375,  500,  375], dtype=float))
velocity_clamp = (-150.0, 150.0)

MAX_ITERS = 50
PATIENCE = 10
pso_runs = []

for seed in PSO_SEEDS:
    set_all_seeds(seed)
    optimizer_pso = pso.GlobalBestPSO(n_particles=20, dimensions=6, options=options, bounds=bounds, velocity_clamp=velocity_clamp)
    
    best_cost, best_pos, stall = None, None, 0
    cost_hist = []

    for it in range(MAX_ITERS):
        cost, pos = optimizer_pso.optimize(pso_objective_function, iters=1, verbose=False)
        cost_hist.append(float(cost))
        if best_cost is None or cost < (best_cost - 1e-3):
            best_cost, best_pos, stall = float(cost), pos, 0
        else:
            stall += 1
        if stall >= PATIENCE: break

    best_sinr_db = float(-best_cost)
    pso_runs.append({"seed": seed, "best_sinr_db": best_sinr_db, "best_pos": np.array(best_pos, dtype=float).tolist(), "cost_history": cost_hist})
    print(f"  -> Semilla {seed:02d}: SINR = {best_sinr_db:.2f} dB ({len(cost_hist)} iters)")

best_run = max(pso_runs, key=lambda r: r["best_sinr_db"])
best_pos_pso_flat = np.array(best_run["best_pos"], dtype=float)

BEST_POS_PSO = {
    "tx1": [float(best_pos_pso_flat[0]), float(best_pos_pso_flat[1]), TX_Z],
    "tx2": [float(best_pos_pso_flat[2]), float(best_pos_pso_flat[3]), TX_Z],
    "tx3": [float(best_pos_pso_flat[4]), float(best_pos_pso_flat[5]), TX_Z]
}
print(f"  => Mejor global: Semilla {best_run['seed']} ({best_run['best_sinr_db']:.2f} dB)")

# =======================================================
# Paso 4: Generar Gráfica de Convergencia PSO
# =======================================================
print("[4/8] Generando gráfica de convergencia PSO...")
fig_conv = plt.figure(figsize=(10, 6))
colores = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
for idx, run in enumerate(pso_runs):
    plt.plot([-c for c in run["cost_history"]], label=f'Semilla {run["seed"]}', color=colores[idx % len(colores)], alpha=0.8, linewidth=2)
plt.xlabel("Iteraciones", fontsize=12); plt.ylabel("SINR Promedio [dB]", fontsize=12); plt.title("Convergencia PSO", fontsize=14)
plt.grid(True, linestyle="--", alpha=0.6); plt.legend()
fig_conv.savefig(os.path.join(RESULTS_DIR, "pso_convergencia.png"), dpi=300); plt.close(fig_conv)

# =======================================================
# Paso 5: Simulación ULA (Posiciones Optimizadas por PSO)
# =======================================================
print("\n[5/8] Evaluando escenario ULA PSO...")
setup_transmitters(scene, BEST_POS_PSO)
omni_avg_sinr_pso, omni_map_pso = compute_sinr_map(scene, rm_solver)
fc_pso, _, _ = functional_coverage(omni_map_pso, thr_db=0.0)
print(f"  -> SINR Promedio: {omni_avg_sinr_pso:.2f} dB | Cobertura (>0dB): {100*fc_pso:.2f}%")

# =======================================================
# Paso 6: Optimización de Ángulos (Bayes)
# =======================================================
print("\n[6/8] Iniciando Optimización Bayesiana (Ángulos)...")
scene.tx_array = mimo_tx_array

def evaluate_sinr_mimo(az1, el1, az2, el2, az3, el3):
    scene.transmitters["tx1"].orientation = [float(az1), float(el1), 0.0]
    scene.transmitters["tx2"].orientation = [float(az2), float(el2), 0.0]
    scene.transmitters["tx3"].orientation = [float(az3), float(el3), 0.0]
    avg_sinr, _ = compute_sinr_map(scene, rm_solver)
    return float(avg_sinr)

pbounds = {
    "az1": (0, 2*np.pi), "el1": (-np.pi/4, np.pi/4),
    "az2": (0, 2*np.pi), "el2": (-np.pi/4, np.pi/4),
    "az3": (0, 2*np.pi), "el3": (-np.pi/4, np.pi/4),
}

# verbose=0 silencia la impresión línea por línea del algoritmo
optimizer_bayes = BayesianOptimization(f=evaluate_sinr_mimo, pbounds=pbounds, random_state=BAYES_SEED, verbose=0)
optimizer_bayes.maximize(init_points=5, n_iter=30)
best_angles = optimizer_bayes.max["params"]
print(f"  => Optimización completada. Mejor SINR proyectado: {optimizer_bayes.max['target']:.2f} dB")

# =======================================================
# Paso 7: Simulación Final (MIMO Optimizado)
# =======================================================
print("\n[7/8] Evaluando escenario MIMO Final...")
scene.transmitters["tx1"].orientation = [float(best_angles["az1"]), float(best_angles["el1"]), 0.0]
scene.transmitters["tx2"].orientation = [float(best_angles["az2"]), float(best_angles["el2"]), 0.0]
scene.transmitters["tx3"].orientation = [float(best_angles["az3"]), float(best_angles["el3"]), 0.0]

mimo_avg_sinr_final, mimo_map_final = compute_sinr_map(scene, rm_solver)
fc_mimo, _, _ = functional_coverage(mimo_map_final, thr_db=0.0)
print(f"  -> SINR Promedio: {mimo_avg_sinr_final:.2f} dB | Cobertura (>0dB): {100*fc_mimo:.2f}%")

# =======================================================
# Paso 8: Guardar Datos .npz
# =======================================================
print("\n[8/8] Guardando datos en formato .npz para gráficas...")
data_file = os.path.join(RESULTS_DIR, "sinr_maps_data.npz")
np.savez(
    data_file,
    omni_map_orig=omni_map_orig, omni_avg_sinr_orig=omni_avg_sinr_orig, fc_orig=fc_orig,
    omni_map_pso=omni_map_pso, omni_avg_sinr_pso=omni_avg_sinr_pso, fc_pso=fc_pso,
    mimo_map_final=mimo_map_final, mimo_avg_sinr_final=mimo_avg_sinr_final, fc_mimo=fc_mimo
)
print(f"  -> {data_file} generado exitosamente.")
print("\n=== Simulación y Optimización Completada ===")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

RESULTS_DIR = "resultados"
DATA_FILE = os.path.join(RESULTS_DIR, "sinr_maps_data.npz")
os.makedirs(RESULTS_DIR, exist_ok=True)
MAP_X_MIN, MAP_X_MAX = -500, 500
MAP_Y_MIN, MAP_Y_MAX = -375, 375
EXTENT = [MAP_X_MIN, MAP_X_MAX, MAP_Y_MIN, MAP_Y_MAX]

# Parámetros de visualización
VMIN_DB, VMAX_DB = -25, 20  
VLIM_DIFF = 15              
VALID_THRESHOLD = -30.0 

try:
    data = np.load(DATA_FILE, allow_pickle=True)
    omni_map_orig = data["omni_map_orig"]
    omni_map_pso = data["omni_map_pso"]
    mimo_map_final = data["mimo_map_final"]
    print("Datos cargados exitosamente.\n")
except FileNotFoundError:
    print(f"Error: El archivo '{DATA_FILE}' no fue encontrado.")
    raise

# *******************************************************
# FIGURA 1 HEATMAPS SINR
# *******************************************************

fig_cov = plt.figure(figsize=(16, 5), constrained_layout=True)
gs_cov = fig_cov.add_gridspec(1, 4, width_ratios=[1.0, 1.0, 1.0, 0.035])
ax1 = fig_cov.add_subplot(gs_cov[0, 0])
ax2 = fig_cov.add_subplot(gs_cov[0, 1])
ax3 = fig_cov.add_subplot(gs_cov[0, 2])
cax_cov = fig_cov.add_subplot(gs_cov[0, 3])
kwargs_cov = dict(origin="lower", extent=EXTENT, cmap="viridis", 
                  vmin=VMIN_DB, vmax=VMAX_DB, aspect="equal", interpolation="nearest")

im1 = ax1.imshow(omni_map_orig, **kwargs_cov)
ax1.set_title("(a) ULA Isotrópica (Original)", fontsize=12)
ax1.set_xlabel("X [m]"); ax1.set_ylabel("Y [m]")

im2 = ax2.imshow(omni_map_pso, **kwargs_cov)
ax2.set_title("(b) ULA + PSO (Posición Optimizada)", fontsize=12)
ax2.set_xlabel("X [m]")

im3 = ax3.imshow(mimo_map_final, **kwargs_cov)
ax3.set_title("(c) MIMO Final (Pos. y Ángulos Opt.)", fontsize=12)
ax3.set_xlabel("X [m]")

cb_cov = fig_cov.colorbar(im3, cax=cax_cov)
cb_cov.set_label("SINR [dB]")

for ax in (ax1, ax2, ax3): ax.grid(True, linewidth=0.3, alpha=0.3)

out_cov = os.path.join(RESULTS_DIR, "1_coberturasinr.png")
fig_cov.savefig(out_cov, dpi=300)
plt.close(fig_cov)
print(f"-> Guardado: {out_cov}")

#******************************************************************************
# FIGURA 2 MAPA DE DIFERENCIAS
#******************************************************************************
diff_pos = omni_map_pso - omni_map_orig
diff_ang = mimo_map_final - omni_map_pso

fig_diff = plt.figure(figsize=(14, 6), constrained_layout=True)
gs_diff = fig_diff.add_gridspec(1, 3, width_ratios=[1.0, 1.0, 0.035])

ax_dp1 = fig_diff.add_subplot(gs_diff[0, 0])
ax_dp2 = fig_diff.add_subplot(gs_diff[0, 1])
cax_diff = fig_diff.add_subplot(gs_diff[0, 2])

kwargs_diff = dict(origin="lower", extent=EXTENT, cmap="bwr", 
                   vmin=-VLIM_DIFF, vmax=VLIM_DIFF, aspect="equal", interpolation="nearest")

im_dp1 = ax_dp1.imshow(diff_pos, **kwargs_diff)
ax_dp1.set_title("(a) Ganancia por Posición (ULA PSO − Original)", fontsize=12)
ax_dp1.set_xlabel("X [m]"); ax_dp1.set_ylabel("Y [m]")

im_dp2 = ax_dp2.imshow(diff_ang, **kwargs_diff)
ax_dp2.set_title("(b) Ganancia por Ángulos (MIMO Final − ULA PSO)", fontsize=12)
ax_dp2.set_xlabel("X [m]")

cb_diff = fig_diff.colorbar(im_dp2, cax=cax_diff)
cb_diff.set_label("ΔSINR [dB]")

for ax in (ax_dp1, ax_dp2): ax.grid(True, linewidth=0.3, alpha=0.3)

out_diff = os.path.join(RESULTS_DIR, "2_diferencias_sinr.png")
fig_diff.savefig(out_diff, dpi=300)
plt.close(fig_diff)
print(f"-> Guardado: {out_diff}")

# *******************************************************
# PREPARACION DE DATOS PARA ESTADISTICAS
# *******************************************************
omni_orig_flat = omni_map_orig.flatten()
omni_pso_flat = omni_map_pso.flatten()
mimo_final_flat = mimo_map_final.flatten()

valid_orig = omni_orig_flat[omni_orig_flat > VALID_THRESHOLD]
valid_pso = omni_pso_flat[omni_pso_flat > VALID_THRESHOLD]
valid_mimo = mimo_final_flat[mimo_final_flat > VALID_THRESHOLD]

# *******************************************************
# FIGURA 3 BOXPLOT
# *******************************************************

fig_box = plt.figure(figsize=(10, 6), constrained_layout=True)
bplot = plt.boxplot(
    [valid_orig, valid_pso, valid_mimo],
    tick_labels=["1. ULA (Original)", "2. ULA (Pos. Opt. PSO)", "3. MIMO (Pos. y Áng. Opt.)"],
    showfliers=False,
    patch_artist=True,
    boxprops=dict(color="black", linewidth=1.2),
    medianprops=dict(color="red", linewidth=2),
    whiskerprops=dict(linewidth=1.2),
    capprops=dict(linewidth=1.2)
)

colors = ['#aec7e8', '#ffbb78', '#98df8a']
for patch, color in zip(bplot['boxes'], colors):
    patch.set_facecolor(color)

plt.ylabel("SINR [dB]", fontsize=12)
plt.title(f"Distribución de SINR (Usuarios con señal > {VALID_THRESHOLD} dB)", fontsize=14)
plt.grid(True, linestyle="--", alpha=0.6)

out_box = os.path.join(RESULTS_DIR, "3_boxplot.png")
fig_box.savefig(out_box, dpi=300)
plt.close(fig_box)
print(f"-> Guardado: {out_box}")

# *******************************************************
# FIGURA 4 Curvas CCDF
# *******************************************************
fig_ccdf = plt.figure(figsize=(10, 6), constrained_layout=True)

def plot_ccdf(dataset, label, color):
    sorted_data = np.sort(dataset)
    y_ccdf = 1.0 - (np.arange(1, len(sorted_data) + 1) / len(sorted_data))
    plt.plot(sorted_data, y_ccdf * 100, label=label, linewidth=2.5, color=color)

plot_ccdf(omni_orig_flat, "1. ULA (Original)", color='#1f77b4')
plot_ccdf(omni_pso_flat, "2. ULA (Pos. Opt. PSO)", color='#ff7f0e')
plot_ccdf(mimo_final_flat, "3. MIMO (Pos. y Áng. Opt.)", color='#2ca02c')

plt.axvline(0, color='gray', linestyle='--', alpha=0.8, label="Umbral Área Útil (0 dB)")
plt.axvline(5, color='gray', linestyle=':', alpha=0.8, label="Umbral Alta Calidad (5 dB)")

plt.xlabel("Umbral de SINR [dB]", fontsize=12)
plt.ylabel("Probabilidad de Cobertura $P(SINR > X)$ [%]", fontsize=12)
plt.title("Función de Distribución Acumulada Complementaria (CCDF)", fontsize=14)
plt.xlim(-20, 25)
plt.ylim(0, 100)
plt.grid(True, alpha=0.5)
plt.legend(fontsize=10, loc="upper right")

out_ccdf = os.path.join(RESULTS_DIR, "4_ccdf_cobertura.png")
fig_ccdf.savefig(out_ccdf, dpi=300)
plt.close(fig_ccdf)
print(f"-> Guardado: {out_ccdf}")

print("\n Generación de gráficas completado")